In [1]:
# Dependencies to pull API
import requests
import json
# Google developer API key
from config import gkey

In [2]:
# Import Dependencies for Database
from config import password
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import psycopg2
import pandas as pd
import time

In [24]:
location_wine_df = pd.read_csv("./Data/US_wine_data.csv")

In [4]:
location_wine_df.columns

Index(['description', 'points', 'price', 'province', 'region_1', 'region_2',
       'title', 'variety', 'winery', 'price_bins'],
      dtype='object')

In [6]:
region_df = location_wine_df.drop(columns = ['description', 'points', 'price', 'province',
       'region_2', 'title', 'variety', 'winery','price_bins'], axis = 1)
region_df = region_df.drop_duplicates()

In [25]:
region_list = list(region_df['region_1'])
region_list

['Willamette Valley',
 'Lake Michigan Shore',
 'Napa Valley',
 'Alexander Valley',
 'Central Coast',
 'Virginia',
 'Oregon',
 'Paso Robles',
 'Sonoma Coast',
 'Clarksburg',
 'Dry Creek Valley',
 'Sonoma Valley',
 'McMinnville',
 'Lake County',
 'Monticello',
 'North Coast',
 'Columbia Valley (WA)',
 'Santa Ynez Valley',
 'California',
 'Howell Mountain',
 'Calistoga',
 'Eola-Amity Hills',
 'Ancient Lakes',
 'Knights Valley',
 'Sonoma County',
 'Finger Lakes',
 'Santa Clara Valley',
 'Yountville',
 'Rockpile',
 'Santa Lucia Highlands',
 'Spring Mountain District',
 'Monterey',
 'Carmel Valley',
 'Santa Cruz Mountains',
 'Edna Valley',
 'Russian River Valley',
 'Arroyo Seco',
 'Anderson Valley',
 'Nevada County',
 'Cayuga Lake',
 'Santa Barbara County',
 'Diamond Mountain District',
 'Mendocino',
 'Sonoma Mountain',
 'Amador County',
 'St. Helena',
 'Cole Ranch',
 'Santa Maria Valley',
 'Dundee Hills',
 'Arroyo Grande Valley',
 'Mendocino County',
 'Rutherford',
 'Walla Walla Valley (WA)

In [26]:
# Target valley
#region = "Sonoma Valley"

# Build the endpoint URL
valley_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?inputtype=textquery&key='+ gkey



In [27]:
regions = str(region_df)
regions.format

<function str.format>

In [28]:
# Create an empty list to hold the region data.
place_id_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [29]:
# Loop through all the regions in the list.
for i, region in enumerate(region_list):

 # Group regions in sets of 25 for logging purposes.
    if (i % 25 == 0 and i >= 25):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each region_id.
    placeid_url = valley_url + '&input='+ region
    

    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    # Log the URL, record, and set numbers and the region.
    print(f"Processing Record {record_count} of Set {set_count} | {region}")
    # Add 1 to the record count.
    record_count += 1 
    
# Run an API request for each of the regions.
    try:
        # Parse the JSON and retrieve data.
        region_url = requests.get(placeid_url).json()
        # Parse out the needed data.
        place_id = region_url['candidates'][0]['place_id']
        # Append the region information into place_id_data list.
        place_id_data.append({"region": region, 
                              "Place_id": place_id})

# If an error is experienced, skip the region.
    except:
        print("region not found. Skipping...")
        pass



# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Processing Record 1 of Set 1 | Willamette Valley
Processing Record 2 of Set 1 | Lake Michigan Shore
Processing Record 3 of Set 1 | Napa Valley
Processing Record 4 of Set 1 | Alexander Valley
Processing Record 5 of Set 1 | Central Coast
Processing Record 6 of Set 1 | Virginia
Processing Record 7 of Set 1 | Oregon
Processing Record 8 of Set 1 | Paso Robles
Processing Record 9 of Set 1 | Sonoma Coast
Processing Record 10 of Set 1 | Clarksburg
Processing Record 11 of Set 1 | Dry Creek Valley
region not found. Skipping...
Processing Record 12 of Set 1 | Sonoma Valley
Processing Record 13 of Set 1 | McMinnville
Processing Record 14 of Set 1 | Lake County
Processing Record 15 of Set 1 | Monticello
Processing Record 16 of Set 1 | North Coast
Processing Record 17 of Set 1 | Columbia Valley (WA)
Processing Record 18 of Set 1 | Santa Ynez Valley
Processing Record 19 of Set 1 | California
Processing Record 20 of Set 1 | Howell Mountain
Processing Record 21 of Set 1 | Calistoga
Processing Record 22

Processing Record 22 of Set 7 | Illinois
Processing Record 23 of Set 7 | Malibu-Newton Canyon
Processing Record 24 of Set 7 | Mount Harlan
Processing Record 25 of Set 7 | Sonoma-Napa-Mendocino
Processing Record 1 of Set 8 | San Francisco Bay
Processing Record 2 of Set 8 | Chelan County
Processing Record 3 of Set 8 | California Other
Processing Record 4 of Set 8 | Saddle Rock-Malibu
Processing Record 5 of Set 8 | Moon Mountain District Sonoma County
Processing Record 6 of Set 8 | Humboldt County
Processing Record 7 of Set 8 | Napa-Sonoma-Marin
region not found. Skipping...
Processing Record 8 of Set 8 | Santa Clara County
Processing Record 9 of Set 8 | Puget Sound
Processing Record 10 of Set 8 | Solano County Green Valley
Processing Record 11 of Set 8 | California-Oregon
Processing Record 12 of Set 8 | Clear Lake
Processing Record 13 of Set 8 | Guenoc Valley
Processing Record 14 of Set 8 | Napa County-Sonoma County-San Joaquin County
Processing Record 15 of Set 8 | Washington-Oregon
Pro

In [30]:
placeid_df = pd.DataFrame(place_id_data)
placeid_df.head(10)

,region,Place_id
0,Willamette Valley,ChIJbezwJP6hlVQRBDL4exvIWSo
1,Lake Michigan Shore,ChIJbWjISvi2EIgR-XXUbfv_U1k
2,Napa Valley,ChIJMU8qI_lPhIARfMUQoxvSFP4
3,Alexander Valley,ChIJD2TljFMUhIARBOZe4vCoE7o
4,Central Coast,ChIJn1wzOELdQIYRJsEGV66URVc
5,Virginia,ChIJzbK8vXDWTIgRlaZGt0lBTsA
6,Oregon,ChIJVWqfm3xuk1QRdrgLettlTH0
7,Paso Robles,ChIJvcTFFo_d7IAR79XMkfgilEY
8,Sonoma Coast,ChIJU2oMa7zAQIYRkEg9eGVDXnE
9,Clarksburg,ChIJd37rCctnSogR4EfMcM4e6V0


In [22]:
place_id = geo_data['candidates'][0]['place_id']
place_id

IndexError: list index out of range

In [ ]:
place_url = ('https://maps.googleapis.com/maps/api/place/details/json?''place_id=ChIJd517gUOshYARReDmDiHtgAM&key={0}').format(gkey)

In [ ]:
place_data = requests.get(place_url).json()
print(place_data)

In [ ]:
latitude = place_data['result']['geometry']['location']['lat']
latitude


In [ ]:
longitude = place_data['result']['geometry']['location']['lng']
longitude